In [2]:
USE_GPU = False
from typing import List, Tuple, Dict, Any, Optional
import seaborn as sns
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
import statsmodels.api as sm
import re
import plotly.express as px
import tensorboard
import pandas as pd
import numpy as np
import seaborn as sns
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import matplotlib.pyplot as plt
import os
import glob
from typing import Dict
from statsmodels.tsa.filters.hp_filter import hpfilter
import re
from datetime import datetime
import plotly
import plotly.offline as pyo
import plotly.io as pio

from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [3]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('mode.chained_assignment', None)


In [4]:
pio.renderers.default = 'notebook+pdf'
# pyo.init_notebook_mode()

In [5]:


TIME_STATS = [
    'sess_time/ended_time',
    'sess_time/session_minutes',
    'sess_time/time_cutoff',
    'sess_time/time_large',
    'sess_time/time_medium',
    'sess_time/time_small',
]

TIME_STATS_GRANULAR = [
    'ended_time', 
    'session_minutes', 
    'time_cutoff', 
    'time_large', 
    'time_medium', 
    'time_small'
]

SIZE_STATS_GRANULAR = [
    'ended_event',
    'session_size',
    'size_cutoff',
    'inc_small',
    'inc_medium',
    'inc_large'
]
    

In [6]:

def plot_vectors(df, x_name, y, title):

    fig = px.line(
        df,
        x=x_name,
        y=y,
        color='model',
        title=f'Training {title}',
    )
    

    return fig
def plot_vectors_multiple(df, y, title, show=True):

    fig = px.line(
        df,
        x='step',
        y=y,
        title=f'Training {title}',
        showlegend=show
    )
    
    # fig.show()
    return fig
    

def global_illegal_moves(df_matrix):
    illegal_move_matrix = {}
    for df_name, df_path in df_matrix.items():
        print(f'Calculating illegal move {df_name} -> {df_path}')
        df = pd.read_csv(df_path)
        df = df[df['exp_runs'] <= 4000]
        df['illegal_move'] = df['ended_event'] == -1
        count_moves = df.groupby(['exp_runs']).size().reset_index(name='exp_run_count')
        illegal_moves = df.groupby(['exp_runs'])['illegal_move'].sum().reset_index(name='illegal_move_count')
        perc_illegal_moves = pd.merge(count_moves, illegal_moves, on='exp_runs')
        perc_illegal_moves['perc_illegal_moves'] = perc_illegal_moves['illegal_move_count'] / perc_illegal_moves['exp_run_count']
        illegal_move_matrix[df_name] = perc_illegal_moves
    
    
    for df_name, df in illegal_move_matrix.items():
        df = df.rolling(20, center=True) \
            .mean() \
            .dropna() \
            .reset_index(drop=True)
        
        df = df[::20]
    
        # cycle, trend = sm.tsa.filters.hpfilter(df['perc_illegal_moves'], lamb=100)
        # df['perc_illegal_moves'] = trend
        df['model'] = df_name
        df[f'q25'] = np.percentile(df['perc_illegal_moves'], 25)
        df[f'q75'] = np.percentile(df['perc_illegal_moves'], 75)
        illegal_move_matrix[df_name] = df
        
    
    out_df = pd.concat(illegal_move_matrix.values())
    out_df = out_df.rename(columns={'exp_runs': 'episode'})
    return out_df 

def global_perc_to_end(df_matrix, session_size_bound=None):
    move_to_end_matrix = {}
    for df_name, df_path in df_matrix.items():
        print(f'Calculating percentage done for {df_name} {df_path}')
        df = pd.read_csv(df_path)
        df = df[df['exp_runs'] <= 4000]
        if session_size_bound:
            df = df[
                (df['session_size'] >= session_size_bound[0]) & (df['session_size'] <= session_size_bound[1])
            ]
       
        df = df[df['ended_event'] > 0]
        df['perc_to_end'] = df['ended_event'] / df['session_size']
        perc_to_end_summary = df.groupby(['exp_runs'])['perc_to_end'].mean().reset_index(name='perc_to_end')
        move_to_end_matrix[df_name] = perc_to_end_summary
        

    for df_name, df in move_to_end_matrix.items():
        df = df.rolling(20, center=True) \
            .mean() \
            .dropna() \
            .reset_index(drop=True)
        
        df = df[::20]
        df['model'] = df_name
        move_to_end_matrix[df_name] = df
        
    out_df = pd.concat(move_to_end_matrix.values())
    out_df = out_df.rename(columns={'exp_runs': 'episode'})
    
    return out_df


def global_perc_done(df_matrix, session_size_bound=None):
    done_matrix = {}
    for df_name, df_path in df_matrix.items():
        print(f'Calculating percentage finished for {df_name} from {df_path}')
        df = pd.read_csv(df_path)
        df = df[df['exp_runs'] <= 4000]
        if session_size_bound:
            df = df[
                (df['session_size'] >= session_size_bound[0]) & (df['session_size'] <= session_size_bound[1])]
        
        df = df[df['ended_event'] > 0]
       
       
        df['perc_done'] = df['ended_event'] == df['session_size']
        perc_done_summary = df.groupby(['exp_runs']).agg(
            done_count=('perc_done', 'sum'),
            sess_count=('perc_done', 'count')
        ) \
        .reset_index()
        perc_done_summary['perc_done'] = perc_done_summary['done_count'] / perc_done_summary['sess_count']
        done_matrix[df_name] = perc_done_summary
    
    # max_episode = min([df['exp_runs'].max() for df in done_matrix.values()])
    for df_name, df in done_matrix.items():
        df = df.rolling(500, center=True) \
            .mean() \
            .dropna() \
            .reset_index(drop=True)
        
        df = df[::500]
        df['model'] = df_name
        done_matrix[df_name] = df
        
    
    out_df = pd.concat(done_matrix.values())
    out_df = out_df.rename(columns={'exp_runs': 'episode'})
    return out_df
    
    

    

PLOTLY_COLORS = plotly.colors.DEFAULT_PLOTLY_COLORS

def plot_granular(df, model_list, showlegend=False):
    plot_list = []
    for model, color in zip(model_list, PLOTLY_COLORS):
        plot_list.append(
            go.Scatter(
                x=df['episode'],
                y=df[model],
                name=model,
                line=dict(color=color),
                showlegend=showlegend
            )
        )
    
    
    return plot_list

                

In [7]:
TB_LOGS = 'dqn_tb'
CSV_LOGS = 'rl_results/dqn_csv'

In [8]:
csv_dirs = {
    'DQN LABEL CNN': 'dqn_csv/train/dqn_label_cnn.csv',
    'DQN PRED CNN': 'dqn_csv/train/dqn_pred_cnn.csv',
    'DQN NONE CNN': 'dqn_csv/train/dqn_none_cnn.csv',
    'DQN PRED MLP': 'dqn_csv/train/dqn_pred_mlp.csv',
    'DQN PRED CNN NO PEN': 'dqn_csv/train/dqn_pred_cnn_no_pen.csv',
    'DQN NONE CNN NO PEN': 'dqn_csv/train/dqn_none_cnn_no_pen.csv',
}

illegal_moves = {k: csv_dirs[k] for k in csv_dirs if 'NO PEN' not in k}


In [9]:
illegal_moves = global_illegal_moves(illegal_moves)



Calculating illegal move DQN LABEL CNN -> dqn_csv/train/dqn_label_cnn.csv
Calculating illegal move DQN PRED CNN -> dqn_csv/train/dqn_pred_cnn.csv
Calculating illegal move DQN NONE CNN -> dqn_csv/train/dqn_none_cnn.csv
Calculating illegal move DQN PRED MLP -> dqn_csv/train/dqn_pred_mlp.csv


In [12]:
illegal_moves

,episode,exp_run_count,illegal_move_count,perc_illegal_moves,model,q25,q75
0,10.5,100.60,99.40,0.988215,DQN LABEL CNN,0.092752,0.811439
20,30.5,100.70,99.30,0.986197,DQN LABEL CNN,0.092752,0.811439
40,50.5,100.25,99.75,0.995050,DQN LABEL CNN,0.092752,0.811439
60,70.5,100.55,99.45,0.989148,DQN LABEL CNN,0.092752,0.811439
80,90.5,100.40,99.60,0.992099,DQN LABEL CNN,0.092752,0.811439
...,...,...,...,...,...,...,...
3900,3910.5,178.60,21.40,0.120222,DQN PRED MLP,0.118689,0.683144
3920,3930.5,179.75,20.25,0.113286,DQN PRED MLP,0.118689,0.683144
3940,3950.5,179.05,20.95,0.117499,DQN PRED MLP,0.118689,0.683144
3960,3970.5,179.95,20.05,0.111924,DQN PRED MLP,0.118689,0.683144


In [18]:
fig = px.line(
    illegal_moves,
    x='episode',
    y='perc_illegal_moves',
    color='model',
)
fig.update_layout(
    width=750,
    height=500,
    xaxis_title='Episode',
    yaxis_title='Percentage of Illegal Moves',
    margin=dict(l=5, r=5, t=5, b=5)
)

fig.show()

pio.write_image(fig, 'rl_plots/q1/glob_illegal_move.png')

# fig

In [21]:
perc_to_end = global_perc_to_end(csv_dirs)
perc_finished = global_perc_done(csv_dirs)

Calculating percentage done for DQN LABEL CNN dqn_csv/train/dqn_label_cnn.csv
Calculating percentage done for DQN PRED CNN dqn_csv/train/dqn_pred_cnn.csv
Calculating percentage done for DQN NONE CNN dqn_csv/train/dqn_none_cnn.csv
Calculating percentage done for DQN PRED MLP dqn_csv/train/dqn_pred_mlp.csv
Calculating percentage done for DQN PRED CNN NO PEN dqn_csv/train/dqn_pred_cnn_no_pen.csv
Calculating percentage done for DQN NONE CNN NO PEN dqn_csv/train/dqn_none_cnn_no_pen.csv
Calculating percentage finished for DQN LABEL CNN from dqn_csv/train/dqn_label_cnn.csv
Calculating percentage finished for DQN PRED CNN from dqn_csv/train/dqn_pred_cnn.csv
Calculating percentage finished for DQN NONE CNN from dqn_csv/train/dqn_none_cnn.csv
Calculating percentage finished for DQN PRED MLP from dqn_csv/train/dqn_pred_mlp.csv
Calculating percentage finished for DQN PRED CNN NO PEN from dqn_csv/train/dqn_pred_cnn_no_pen.csv
Calculating percentage finished for DQN NONE CNN NO PEN from dqn_csv/trai

In [22]:
perc_to_end

,episode,perc_to_end,model
0,23.9,0.629985,DQN LABEL CNN
20,72.9,0.628659,DQN LABEL CNN
40,137.9,0.622852,DQN LABEL CNN
60,174.4,0.633031,DQN LABEL CNN
80,203.6,0.611117,DQN LABEL CNN
...,...,...,...
3900,3910.5,0.704250,DQN NONE CNN NO PEN
3920,3930.5,0.703290,DQN NONE CNN NO PEN
3940,3950.5,0.698318,DQN NONE CNN NO PEN
3960,3970.5,0.698636,DQN NONE CNN NO PEN


In [24]:
fig = px.line(
    perc_to_end,
    x='episode',
    y='perc_to_end',
    color='model'
)

fig.update_layout(
    width=750, height=500,
    xaxis_title='Episode',
    yaxis_title='Simulation Index / Session Size',
    margin=dict(l=5, r=5, t=5, b=5),
)
pio.write_image(fig, 'rl_plots/q1/glob_perc_to_end.png')


In [25]:
fig = px.line(
    perc_finished,
    x='episode',
    y='perc_done',
    color='model'
)
fig.update_layout(
    width=750, 
    height=500,
    xaxis_title='Episode',
    yaxis_title='Sessions Completed (%)',
    margin=dict(l=5, r=5, t=5, b=5),
)
pio.write_image(fig, 'rl_plots/q1/glob_perc_done.png')

fig